In [245]:
class tree():
            
    def __init__(self, branch_len=None, seq_leaf=None,node_name=None,parent=None,child=None, nuc_list=None,lkl_list=None, type=None):
        self.seq_leaf = seq_leaf
        self.child = child
        self.parent=parent
        self.branch_length = branch_len
        self.nuc_list=nuc_list
        self.lkl_list=lkl_list
        self.name=node_name
        self.type=type

    def __len__(self):
        return len(self.seq_leaf)
    
    def getSequence(self):
        return self.seq_leaf
    
    def getBranchLength(self):
        return self.branch_length
       
    def getChildrenNodes(self):
        return self.child
    
    def get_nuc_list(self):
        return self.nuc_list
    
    def get_lkl_list(self):
        return self.lkl_list
    
    def initialize_nuc_list(self):
        if self.seq_leaf:
            bases={"A":[1,0,0,0],
                "C":[0,1,0,0],
                "G":[0,0,1,0],
                "T":[0,0,0,1]}
            self.nuc_list=list()
            for base in self.seq_leaf:
                self.nuc_list.append(bases[base])
            return self.nuc_list
        else: return None
    



In [246]:
branch_lengths_path="data/branchlength.dat"
tree_path="data/table.dat"
msa_path="data/msa.dat"



with open(tree_path,"r") as tree_file, open(branch_lengths_path,"r") as BL_file, open(msa_path,"r") as msa_file:
    node_added_tmp=[] ## list with nodes added to trr to upadte children
    Tree_Data=[]
    line_num=0
    
    bl=BL_file.readline().split(",")
    seq_for_leaves = {}
    for line in msa_file:
       (key, val) = line.split()
       seq_for_leaves[int(key)] = val


    for parent_child in tree_file:
        tmp_list=parent_child.split(",")
        parent_node_name=tmp_list[0]
 
        child_node_name=tmp_list[1].rstrip()
        branch_len=bl[line_num]
        if int(child_node_name) in seq_for_leaves.keys():
            leaf_seq=seq_for_leaves[int(child_node_name)]
            type="external"
        else:
            leaf_seq=None
            type="internal"
        new_node = tree(branch_len=branch_len,
            seq_leaf=leaf_seq,
            type=type,
            node_name=child_node_name,
            parent=parent_node_name,
            child=child_node_name,
            nuc_list=[],
            lkl_list=[])
        
        
        Tree_Data.append(new_node)
        line_num+=1
    


    


In [247]:
for obj in Tree_Data:
    print("node type    : " , obj.type)
    print("Parent name  : ",obj.parent)
    print("current name : ",obj.name)
    print("Child name   : ",obj.child)
    print("Branch length: ",obj.branch_length)
    print("sequence     : ", obj.seq_leaf)
    print("nuc list     : ", obj.initialize_nuc_list())
    print("nuc_list updated: " , obj.get_nuc_list())
    print("lkl_list updated: " , obj.get_lkl_list())


node type    :  external
Parent name  :  9
current name :  1
Child name   :  1
Branch length:  0.1
sequence     :  AGATCAAGATCAAGATCAAGATCAAGATCA
nuc list     :  [[1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0]]
nuc_list updated:  [[1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1

In [32]:

import numpy as np
import scipy
from scipy.linalg import expm, sinm, cosm
Q=np.array([[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]])


#MAIN PROGRAMME
#we should have branch lengths, sequences (not all nodes ahve sequences--> only the leaf nodes) , tree in our class data structure

# this dict will store each node and the probabilities of 4 nucleotides at each position
# dict = {key is node name: values are list of lists}
#lkl_dict =  {node_name:nuc_likelihood}  
lkl_dict=dict() 

for node in Tree_Data:
    if node.getSequence() is not None: # if it is not internal node 
        nuc_list=node.get_nuc_list() # which nuc is present at each pposition --> a list of lists like [[1,0,0,0],[0,1,0,0]....]
        Probability_Matrix = scipy.linalg.expm(Q * float(node.getBranchLength())) # same for all positions
        for pos_list in nuc_list: # for each pos we have a list like [1,0,0,0] in nuc_list
            nuc_lkl = numpy.matmul(Probability_Matrix, pos_list ) # calculate liklihood of each nucleotide
            merge nuc_lkl to lkl_dict[node] # add to dict
   
    # if it is an internal node:
    else:
        child_names=node.get_children() # get_children will return a list of child node names --> two children per parent
        childA_lkl = lkl_dict[child_names[0]] # we will get a list of lists with prob for each nuclleotide at each position
        childB_lkl = lkl_dict[child_names[1]]
        for pos_list  in range(len(childA_lkl)): #using all positions in both children nodes, calculate ancetral nucleotide liklihoods 
            joint_lkl_AB = [childA_lkl[pos_list ][i] * childB_lkl[pos_list ][i] for i in range(4)]
            anc_lkl= numpy.matmul(joint_lkl_AB, [0.25,0.25,0.25,0.25] ) # all nucleotides ahve sam eprobabilities
            merge anc_lkl to lkl_dict[node]
return sum(math.log(anc_lkl)) # values in npmm will be the ones for root

SyntaxError: invalid syntax (2261233279.py, line 21)

In [248]:
import numpy as np
import scipy
from scipy.linalg import expm, sinm, cosm
Q=np.array([[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]])


lkl_dict = {}

for node in Tree_Data:
    if node.getSequence() is not None: # if it is not internal node 
        nuc_list=node.get_nuc_list() # which nuc is present at each pposition --> a list of lists like [[1,0,0,0],[0,1,0,0]....]
        Probability_Matrix = scipy.linalg.expm(Q * float(node.getBranchLength())) # same for all positions
        #print(Probability_Matrix)
        for pos_list in nuc_list:
            node.lkl_list.append(np.matmul(Probability_Matrix,pos_list))
        lkl_dict[node.name]=node.lkl_list
        #print(node.name,node.child,node.lkl_list)
            #print(node.get_lkl_list())



for node in lkl_dict:
    print("dict", node ," ",lkl_dict[node])


dict 1   [array([1.12295617, 0.12295617, 0.12295617, 0.12295617]), array([0.12295617, 0.12295617, 1.12295617, 0.12295617]), array([1.12295617, 0.12295617, 0.12295617, 0.12295617]), array([0.12295617, 0.12295617, 0.12295617, 1.12295617]), array([0.12295617, 1.12295617, 0.12295617, 0.12295617]), array([1.12295617, 0.12295617, 0.12295617, 0.12295617]), array([1.12295617, 0.12295617, 0.12295617, 0.12295617]), array([0.12295617, 0.12295617, 1.12295617, 0.12295617]), array([1.12295617, 0.12295617, 0.12295617, 0.12295617]), array([0.12295617, 0.12295617, 0.12295617, 1.12295617]), array([0.12295617, 1.12295617, 0.12295617, 0.12295617]), array([1.12295617, 0.12295617, 0.12295617, 0.12295617]), array([1.12295617, 0.12295617, 0.12295617, 0.12295617]), array([0.12295617, 0.12295617, 1.12295617, 0.12295617]), array([1.12295617, 0.12295617, 0.12295617, 0.12295617]), array([0.12295617, 0.12295617, 0.12295617, 1.12295617]), array([0.12295617, 1.12295617, 0.12295617, 0.12295617]), array([1.12295617, 0.

In [240]:
# Separating external and internal nodes
nodes_with_sequence = []
nodes_without_sequence = []

for node in Tree_Data:
    if node.getSequence() is not None:
        nodes_with_sequence.append(node)
    else:
        nodes_without_sequence.append(node)

# Loop through objects with sequences
for node_with_seq in nodes_with_sequence:
    # Perform operations or access attributes for objects with sequences
    print(f"Object with sequence: {node_with_seq.name}")
    

# Loop through objects without sequences
for node_without_seq in nodes_without_sequence:
    # Perform operations or access attributes for objects without sequences
    print(f"Object without sequence: {node_without_seq.name}")

Object with sequence: 1
Object with sequence: 2
Object with sequence: 3
Object with sequence: 4
Object with sequence: 5
Object without sequence: 9
Object without sequence: 7
Object without sequence: 8


In [273]:

parent_nodes_visited=[]

for nodeA in Tree_Data:
    for nodeB in Tree_Data:
        if nodeA != nodeB:
            if nodeA.parent == nodeB.parent and not (nodeA.parent in parent_nodes_visited):

                childA_lkl = lkl_dict[nodeA.name] # we will get a list of lists with prob for each nuclleotide at each position
                childB_lkl = lkl_dict[nodeB.name]

                anc_AB_lkl=[]
                anc_name=nodeA.parent

                for pos_list  in range(len(childA_lkl)): #using all positions in both children nodes, calculate ancetral nucleotide liklihoods 
                    vec1=childA_lkl[pos_list]
                    vec2=childB_lkl[pos_list]
                    vec_anc=  [vec1[i] * vec2[i] for i in range(4)]

                    print("node",nodeA.parent,vec_anc)


                    ### WHY THIS RETURN ONLY ONE NUMBER INSTEAD OF 4?????????
                    vec_anc=np.matmul(vec_anc, [0.25,0.25,0.25,0.25]) # all nucleotides ahve sam eprobabilities

                    print("node",nodeA.parent,vec_anc)
                    
                    break
                    #anc_AB_lkl.append(np.array(vec_anc))

                #break
                lkl_dict[anc_name]=anc_AB_lkl
                parent_nodes_visited.append(nodeA.parent)
               # sum(math.log(anc_AB_lkl))




                

                

node 9 [2.2327267165649873, 0.1215124360558915, 0.12151243605589147, 0.12151243605589152]
node 9 0.6493160061831654
node 7 [0.028886594404036094, 1.4295537676111423, 0.02888659440403609, 0.028886594404036094]
node 7 0.3790533877058127


In [258]:
lkl_dict


{'1': [array([1.12295617, 0.12295617, 0.12295617, 0.12295617]),
  array([0.12295617, 0.12295617, 1.12295617, 0.12295617]),
  array([1.12295617, 0.12295617, 0.12295617, 0.12295617]),
  array([0.12295617, 0.12295617, 0.12295617, 1.12295617]),
  array([0.12295617, 1.12295617, 0.12295617, 0.12295617]),
  array([1.12295617, 0.12295617, 0.12295617, 0.12295617]),
  array([1.12295617, 0.12295617, 0.12295617, 0.12295617]),
  array([0.12295617, 0.12295617, 1.12295617, 0.12295617]),
  array([1.12295617, 0.12295617, 0.12295617, 0.12295617]),
  array([0.12295617, 0.12295617, 0.12295617, 1.12295617]),
  array([0.12295617, 1.12295617, 0.12295617, 0.12295617]),
  array([1.12295617, 0.12295617, 0.12295617, 0.12295617]),
  array([1.12295617, 0.12295617, 0.12295617, 0.12295617]),
  array([0.12295617, 0.12295617, 1.12295617, 0.12295617]),
  array([1.12295617, 0.12295617, 0.12295617, 0.12295617]),
  array([0.12295617, 0.12295617, 0.12295617, 1.12295617]),
  array([0.12295617, 1.12295617, 0.12295617, 0.1229

In [201]:
for obj_with_seq in objects_with_sequence:
    # Perform operations or access attributes for objects without sequences
    print(f"Object without sequence: {obj_with_seq.name, obj_with_seq.parent, obj_with_seq.child}")


lkl_dict.keys()

Object without sequence: ('1', '9', '1')
Object without sequence: ('2', '9', '2')
Object without sequence: ('3', '8', '3')
Object without sequence: ('4', '7', '4')
Object without sequence: ('5', '7', '5')


dict_keys(['1', '2', '3', '4', '5'])